In [12]:
import csv
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from utils import *
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import tensorflow as tf
import tensorflowjs as tfjs
from preprocess import Preprocessor
from train import *

In [3]:
train= make_df("dataset", "train")
train.head

  0%|          | 0/816 [00:00<?, ?it/s]

100%|██████████| 324/324 [00:03<00:00, 92.53it/s]

['Skipped 0.jpg:Zone.Identifier - Invalid file type', 'Skipped 1.jpg:Zone.Identifier - Invalid file type', 'Skipped 10.jpg:Zone.Identifier - Invalid file type', 'Skipped 11.jpg:Zone.Identifier - Invalid file type', 'Skipped 12.jpg:Zone.Identifier - Invalid file type', 'Skipped 13.jpg:Zone.Identifier - Invalid file type', 'Skipped 14.jpg:Zone.Identifier - Invalid file type', 'Skipped 15.jpg:Zone.Identifier - Invalid file type', 'Skipped 16.jpg:Zone.Identifier - Invalid file type', 'Skipped 17.jpg:Zone.Identifier - Invalid file type', 'Skipped 18.jpg:Zone.Identifier - Invalid file type', 'Skipped 19.jpg:Zone.Identifier - Invalid file type', 'Skipped 2.jpg:Zone.Identifier - Invalid file type', 'Skipped 20.jpg:Zone.Identifier - Invalid file type', 'Skipped 21.jpg:Zone.Identifier - Invalid file type', 'Skipped 22.jpg:Zone.Identifier - Invalid file type', 'Skipped 23.jpg:Zone.Identifier - Invalid file type', 'Skipped 24.jpg:Zone.Identifier - Invalid file type', 'Skipped 25.jpg:Zone.Identifie

<bound method NDFrame.head of     filename  NOSE_x  NOSE_y  NOSE_score  LEFT_EYE_x  LEFT_EYE_y  \
0      0.jpg    61.0    58.0         1.0        56.0        54.0   
1      1.jpg    60.0    54.0         1.0        54.0        51.0   
2     10.jpg    98.0    55.0         1.0        93.0        50.0   
3     11.jpg   114.0    67.0         1.0       111.0        62.0   
4     12.jpg   118.0    80.0         1.0       114.0        75.0   
..       ...     ...     ...         ...         ...         ...   
666   95.jpg    63.0    27.0         1.0        68.0        23.0   
667   96.jpg    68.0    26.0         1.0        72.0        23.0   
668   97.jpg    74.0    29.0         1.0        77.0        24.0   
669   98.jpg    81.0    30.0         1.0        84.0        26.0   
670   99.jpg    86.0    30.0         1.0        89.0        26.0   

     LEFT_EYE_score  RIGHT_EYE_x  RIGHT_EYE_y  RIGHT_EYE_score  ...  \
0               1.0         61.0         54.0              1.0  ...   
1          

In [4]:
X, y, class_names = load_data(train)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# Second split: validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [5]:
processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)
# print("process x train", processed_X_train)
# print("process x test", processed_X_test)

inputs = tf.keras.Input(shape=(34,))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)


In [10]:
model = keras.Model(inputs, outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
# Update checkpoint path to use the .keras extension
checkpoint_path = "weights.best.keras"

checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                             monitor='val_accuracy',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='max')

earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)


In [13]:
# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=500,
                    batch_size=32,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])


print('-----------------EVALUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)

tfjs_model_dir = 'model'
tfjs.converters.save_keras_model(model, tfjs_model_dir)
print('tfjs model saved at ',tfjs_model_dir)



--------------TRAINING----------------
Epoch 1/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0323
Epoch 1: val_accuracy did not improve from 1.00000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9927 - loss: 0.0296 - val_accuracy: 1.0000 - val_loss: 0.0095
Epoch 2/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.0243
Epoch 2: val_accuracy did not improve from 1.00000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9942 - loss: 0.0313 - val_accuracy: 1.0000 - val_loss: 0.0091
Epoch 3/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0145
Epoch 3: val_accuracy did not improve from 1.00000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9942 - loss: 0.0207 - val_accuracy: 1.0000 - val_loss: 0.0083
Epoch 4/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.0152
Epoch 4: val_accuracy did not improve from 1.00000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9975 - loss: 0.

failed to lookup keras version from the file,
    this is likely a weight only file
tfjs model saved at  model


## TRAIN WITH ANGLES

processed_X_train = preprocess_data_with_angles(X_train)
processed_X_val =  preprocess_data_with_angles(X_val)
processed_X_test = preprocess_data_with_angles(X_test)
# print("process x train", processed_X_train)
# print("process x test", processed_X_test)

inputs = tf.keras.Input(shape=(40,))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)


In [14]:
model = keras.Model(inputs, outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
# Update checkpoint path to use the .keras extension
checkpoint_path = "weights.angles.keras"

checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                             monitor='val_accuracy',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='max')

earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)


In [15]:
# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=500,
                    batch_size=32,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])


print('-----------------EVALUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)


tfjs.converters.save_keras_model(model, tfjs_model_dir)
print('tfjs model saved at ',tfjs_model_dir)


--------------TRAINING----------------
Epoch 1/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 8s 618ms/step - accuracy: 1.0000 - loss: 0.0089
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to weights.angles.keras
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9977 - loss: 0.0175 - val_accuracy: 1.0000 - val_loss: 0.0057
Epoch 2/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.0095
Epoch 2: val_accuracy did not improve from 1.00000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9994 - loss: 0.0141 - val_accuracy: 1.0000 - val_loss: 0.0031
Epoch 3/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0461
Epoch 3: val_accuracy did not improve from 1.00000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9967 - loss: 0.0237 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 4/500
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0174
Epoch 4: val_accuracy did not improve from 1.00000
15/15 ━━━━━━━━━━━━━━━━━━━━ 

LOSS:  0.0005782356602139771
ACCURACY:  1.0
failed to lookup keras version from the file,
    this is likely a weight only file
tfjs model saved at  model
